## 1. 할당된 GPU 확인

현재 런타임에 할당된 GPU를 확인합니다.

만약, `GPU: NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.` 라는 메시지가 출력된다면, 위쪽 메뉴에서 `런타임 -> 런타임 유형 변경`을 클릭하고 하드웨어 가속기를 `GPU`로 변경하여 저장한 후 다시 실행해주세요.

In [1]:
import os
GPU_NAME = os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read().strip()
os.environ['GPU_NAME'] = GPU_NAME
print(f'GPU: {GPU_NAME}')

GPU: Tesla V100-PCIE-32GB


## 3. 필수 라이브러리 및 함수 불러오기

실행에 필요한 라이브러리 및 함수를 불러옵니다.

이 과정은 약 10분 정도 소요될 수 있습니다.

In [2]:
import os
os.chdir('../../../')
current_dir = os.getcwd()
print(current_dir)

/opt/ml


In [3]:
import sys
from pathlib import Path

In [4]:
# 가상 환경 생성
!python3 -m venv tts-env

# 가상 환경 활성화
!source tts-env/bin/activate

# numpy 업그레이드
!pip install --upgrade numpy

# TTS 설치
!pip install TTS==0.0.14.1


  Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.0.14.1 requires numpy==1.18.5, but you have numpy 1.24.4 which is incompatible.
sagemaker-training 4.5.0 requires protobuf<=3.20.3,>=3.9.2, but you have protobuf 4.23.3 which is incompatible.
  Using cached numpy-1.18.5-cp38-cp38-manylinux1_x86_64.whl (20.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depende

In [5]:
os.chdir('content')
!git clone --depth 1 https://github.com/sce-tts/TTS.git -b sce-tts
os.chdir('TTS')
!python setup.py develop

fatal: destination path 'TTS' already exists and is not an empty directory.
running develop
-- Building version 0.0.14.1
running egg_info
writing TTS.egg-info/PKG-INFO
writing dependency_links to TTS.egg-info/dependency_links.txt
writing entry points to TTS.egg-info/entry_points.txt
writing requirements to TTS.egg-info/requires.txt
writing top-level names to TTS.egg-info/top_level.txt
reading manifest file 'TTS.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'TTS.egg-info/SOURCES.txt'
running build_ext
copying build/lib.linux-x86_64-3.8/TTS/tts/layers/glow_tts/monotonic_align/core.cpython-38-x86_64-linux-gnu.so -> TTS/tts/layers/glow_tts/monotonic_align
Creating /opt/conda/lib/python3.8/site-packages/TTS.egg-link (link to .)
TTS 0.0.14.1 is already the active version in easy-install.pth
Installing tts script to /opt/conda/bin
Installing tts-server script to /opt/conda/bin

Installed /opt/ml/content/TTS
Processing dependencies for TTS==0.0.14.1
Search

## 4. 학습할 데이터셋 불러오기

학습에 사용할 음성 데이터를 구글 드라이브에서 가져옵니다.

In [6]:
#%cd /content/TTS
!cp "../../content/drive/My Drive/Colab Notebooks/data/filelists.zip" ./filelists.zip
!rm -rf ./filelists
!unzip -q filelists.zip -d ./filelists

## 5. 사전 학습 데이터 불러오기

사전 학습 데이터가 구글 드라이브에 존재하지 않을 경우,  
SleepingCE Speech Dataset의 사전 학습 데이터를 내려받습니다.

In [7]:
#%cd /content/TTS
!mkdir -p "../../content/drive/My Drive/Colab Notebooks/data/hifigan-v2"
if not Path("../../content/drive/My Drive/Colab Notebooks/data/hifigan-v2/config.json").exists():
    !gdown --id 1vRxp1RH-U7gSzWgyxnKY4h_7pB3tjPmU -O hifigan-v2.zip
    !unzip -q hifigan-v2.zip -d ./
    !cp -R ./hifigan-v2/* "../../content/drive/My Drive/Colab Notebooks/data/hifigan-v2/"

In [8]:
#%cd /content/TTS
if not Path("../../content/drive/My Drive/Colab Notebooks/data/hifigan-v2/scale_stats_new.npy").exists():
    !python TTS/bin/compute_statistics.py "../../content/drive/My Drive/Colab Notebooks/data/hifigan-v2/config.json" "../../content/drive/My Drive/Colab Notebooks/data/hifigan-v2/scale_stats_new.npy" --data_path "../../content/TTS/filelists/wavs/"

## 7. HiFi-GAN 학습 진행

실제 HiFi-GAN 학습을 진행합니다.

학습이 정상적으로 진행되면, 이 셀은 종료되지 않고 계속 실행되는 상태를 유지합니다.

학습 진행 정도를 시각적으로 확인하시려면 바로 위의 TensorBoard를 참고해주세요.

학습된 결과를 확인하고 싶으시다면, 가장 최근 체크포인트를 [SCE-TTS: 음성합성 데모](https://colab.research.google.com/drive/1YkxjzBz3V4eXoAaEgcFNEUg8ZyWV40x9)에서 불러와 확인할 수 있습니다.

처음 학습을 수행할 때에는 아래 셀을 그대로 실행하면 됩니다.

이전에 학습을 진행하던 모델을 이어서 학습을 진행하시려면 다음과 같이 수정한 후 실행해주세요.

- 아래 셀에서 2 ~ 3번째 줄의 코드를 주석을 해제합니다.
- 3번째 줄의 경로를 이어서 학습을 진행할 모델의 경로로 변경합니다.  
(예시: `/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/hifigan-v2-May-31-2021_08+26AM-d897f2e`)
- 4번째 줄 아래의 코드를 제거합니다.

In [9]:
#%cd /content/TTS
!(python TTS/bin/train_vocoder_gan.py \
    --continue_path "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/hifigan-v2-June-26-2023_07+23AM-3aa165ae")


'''
!(python TTS/bin/train_vocoder_gan.py \
    --config_path "../../content/drive/My Drive/Colab Notebooks/data/hifigan-v2/config.json" \
    --coqpit.data_path "../../content/TTS/filelists/wavs"  \
    --coqpit.audio.stats_path "../../content/drive/My Drive/Colab Notebooks/data/hifigan-v2/scale_stats_new.npy"  \
    --coqpit.output_path "../../content/drive/My Drive/Colab Notebooks/data/hifigan-v2/"  \
    --coqpit.num_loader_workers 2  \
    --coqpit.num_val_loader_wo567890rkers 2  \
    --restore_path "../../content/drive/My Drive/Colab Notebooks/data/hifigan-v2/model_file.pth.tar")
'''

 > Using CUDA:  True
 > Number of GPUs:  1
Traceback (most recent call last):
  File "TTS/bin/train_vocoder_gan.py", line 626, in <module>
    args, c, OUT_PATH, AUDIO_PATH, c_logger, tb_logger = init_training(sys.argv)
  File "/opt/ml/content/TTS/TTS/utils/arguments.py", line 181, in init_training
    config, OUT_PATH, AUDIO_PATH, c_logger, tb_logger = process_args(args)
  File "/opt/ml/content/TTS/TTS/utils/arguments.py", line 141, in process_args
    args.restore_path, best_model = get_last_checkpoint(args.continue_path)
  File "/opt/ml/content/TTS/TTS/utils/arguments.py", line 107, in get_last_checkpoint
    raise ValueError(f"No models found in continue path {path}!")
ValueError: No models found in continue path /content/drive/My Drive/Colab Notebooks/data/hifigan-v2/hifigan-v2-June-26-2023_07+23AM-3aa165ae!


'\n!(python TTS/bin/train_vocoder_gan.py     --config_path "../../content/drive/My Drive/Colab Notebooks/data/hifigan-v2/config.json"     --coqpit.data_path "../../content/TTS/filelists/wavs"      --coqpit.audio.stats_path "../../content/drive/My Drive/Colab Notebooks/data/hifigan-v2/scale_stats_new.npy"      --coqpit.output_path "../../content/drive/My Drive/Colab Notebooks/data/hifigan-v2/"      --coqpit.num_loader_workers 2      --coqpit.num_val_loader_wo567890rkers 2      --restore_path "../../content/drive/My Drive/Colab Notebooks/data/hifigan-v2/model_file.pth.tar")\n'